# Google API Explorations  
---

## Objective
This notebook will use the Google API to generate directions that avoid a current closure.

This method may be viable, but requires more data and more tweaking. 

We would need to know whether or not Google is already taking twitter-reported closures into account while routing, and we need to test whether a working location extraction technique would work.

This notebook shows an example of a how we could use Google APIs to convert closures into polyline codes and test whether generated routes include those polyline encodings.

---

In [1]:
import requests
import json
import csv
import polyline

In [2]:
credential_file = '../creds/google-maps-api.json'

with open(credential_file, 'r') as f:
    credentials = json.load(f)
    API_KEY = credentials['API_KEY']

# Google Places API
The Google Places API can take a string and return a specified location json for its most likely match.

In [3]:
def generate_place(location,key=credentials['API_KEY']):
    '''
    Returns a Google Places json object based on the input `location`.
    
    Parameters
    ----------
    location (str): location of place as a string. The Maps API uses Google Search to find the location.
    
    key (str): credentials['API_KEY']
    
    Returns
    -------
    place (json): Returns a json object with place information.
    '''
    # Format the location objeect to pass into the API request.
    location = location.lower().replace(' ','+')
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={location}&key={key}"
    res = requests.get(url)
    return res.json()

In [7]:
generate_place('Pepperdine University')

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}

# Google Directions API
The Directions API can take origins and destinations and give you routes between them, including alternate routes. The json includes the steps for each route as well.

In [94]:
def generate_directions(origin,destination,key=credentials['API_KEY']):
#     orig = origin['results']['name'].lower().replace(' ','+')
#     dest = destination['results']['name'].lower().replace(' ','+')
    url = f"https://maps.googleapis.com/maps/api/directions/json?" \
            f"origin={origin.lower().replace(' ','+')}&" \
            f"destination={destination.lower().replace(' ','+')}&" \
            f"alternatives=true&" \
            f"key={key}"
    return requests.get(url).json()

In [101]:
directions = generate_directions('Pepperdine University','The Getty Museum')
directions

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJp1Mqvq8f6IARk9BdNpkiaO4',
   'types': ['establishment', 'point_of_interest', 'school', 'university']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJbzYnQte8woARJaqqFVpKeNo',
   'types': ['establishment',
    'museum',
    'point_of_interest',
    'tourist_attraction']}],
 'routes': [{'bounds': {'northeast': {'lat': 34.08872540000001,
     'lng': -118.4357197},
    'southwest': {'lat': 34.0113659, 'lng': -118.7035686}},
   'copyrights': 'Map data ©2019',
   'legs': [{'distance': {'text': '21.8 mi', 'value': 35162},
     'duration': {'text': '36 mins', 'value': 2146},
     'end_address': '1200 Getty Center Dr, Los Angeles, CA 90049, USA',
     'end_location': {'lat': 34.0882892, 'lng': -118.4763226},
     'start_address': '24255 Pacific Coast Hwy, Malibu, CA 90263, USA',
     'start_location': {'lat': 34.0393452, 'lng': -118.7020505},
     'steps': [{'distance': {'text': '174 ft', 'value': 53},
       'duration': {'

In [113]:
directions['routes'][0]['summary']

'CA-1 S'

# Polyline Encoder/Decoder
Google returns lists of longitude and latitude coordinates encoded as a polyline. Using the `polyline` package in python, one can decode a list of points along a given route. We will use these points to check if a route is passing through a "closed polyline".

In [96]:
# An example of Polyline Encoding
polyline.encode([(38.5, -120.2), (40.7, -120.9), (43.2, -126.4)], 5)

'_p~iF~ps|U_ulL~ugC_hgN~eq`@'

In [97]:
# An Example of polyline decoding.
polyline.decode('_p~iF~ps|U_ulL~ugC_hgN~eq`@')

[(38.5, -120.2), (40.7, -120.9), (43.2, -126.4)]

In [100]:
# We can generate a list of longitude and latitude strings 
# by extracting each polyline string from each step of the route.
def get_lat_long(route):
    lat_long = []
    for step in route['legs'][0]['steps']:
        lat_long += polyline.decode(step['polyline']['points'])
    return lat_long
    

# Avoiding Road Closures
We need to get a road closure longitude and latitude start and end point.
Steps:
1. Convert closure to a passable bit of data.
    - Create route and collect `lat_long points`
2. Check if the closure is included in the given routes.
    - **If not:** Give the shortest and best route.
    - **If so:** Continue with algorithm.
3. Check alternate routes to see if they go through the road closure and select one if it does not.
4. If there are no alternate routes, create a series of way points in a Fibonacci spiral radiating out from the beginning of the closure (or maybe the starting point) until a suitable alternate route is found.

## Example of Rerouting 
Suppose there is a patch of road on our route between Pepperdine Univeristy and the Getty Museum that is closed. Let's specify it as the route between the [McDonald's](google.com/maps/place/McDonald's/@34.1481745,-118.683656,15z/data=!4m5!3m4!1s0x80e820c71a80784b:0xe24965b370f8c77f!8m2!3d34.144303!4d-118.6986145) and a nearby [Mercedes Benz dealership](https://www.google.com/maps/place/Mercedes-Benz+of+Calabasas/@34.1481745,-118.683656,15z/data=!4m5!3m4!1s0x80c29e14231e2b99:0x1352a76899a3293e!8m2!3d34.1521366!4d-118.6537653). 

  
![malibu_route](../images/malibu_route.png)

In [103]:
get_lat_long(directions['routes'][0])[:10]

[(34.03935, -118.70205),
 (34.03943, -118.70216),
 (34.03953, -118.70229),
 (34.03958, -118.70234),
 (34.0397, -118.70244),
 (34.0397, -118.70244),
 (34.03965, -118.70253),
 (34.03959, -118.70248),
 (34.03945, -118.70234),
 (34.03944, -118.70232)]

In [107]:
closure = generate_directions("McDonald's, 4785 Las Virgenes Rd, Calabasas, CA 91302","Mercedes-Benz of Calabasas, 24181 Calabasas Rd, Calabasas, CA 91302")
lat_long_closure = get_lat_long(closure['routes'][0])
lat_long_closure[:10]

[(34.14417, -118.6985),
 (34.14418, -118.69848),
 (34.14419, -118.69847),
 (34.1442, -118.69845),
 (34.1442, -118.69843),
 (34.14421, -118.69839),
 (34.14421, -118.69838),
 (34.14421, -118.69816),
 (34.14421, -118.69816),
 (34.14429, -118.69815)]

In [120]:
def check_routes(routes):
    good_routes = []
    for route in routes:
        route_good = True
        for point in lat_long_closure:
            if point in get_lat_long(route):
                route_good = False
        if route_good:
            good_routes.append(route)
            print(f"OK to use {route['summary']} route")
        else:
            print(f"Avoid {route['summary']} Route.")
    return good_routes

In [122]:
check_routes(directions['routes']);

OK to use CA-1 S route
Avoid US-101 S Route.
OK to use CA-1 S and San Vicente Blvd route


> Notice that our rerouter tells us to take the coastal route, rather than trying to take the US-101 Route. It works!

# Summary
As you can see, this algorithm works in checking if routes are passing through known closures. If we could compile a list of "no pass waypoints" from our tweets, and if we could check if the closure was already being accounted for by Google, this might be a viable option.

Iterations would also need to include a method for "forcing" Google Directions to generate more alternatives. One method would be to introduce waypoints to the API request in a spiral pattern with increasing distance to make the Directions API generate routes that take the user away from the closed area.